In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import time
import keras

In [2]:
df = pd.read_csv("../data/customer_service/twcs/twsc_sorted.csv", parse_dates=[3])

In [3]:
amazon = df[df.author_id == 'AmazonHelp']

In [4]:
def loadGloveModel(File):
    tic = time.time()
    print("Loading Glove Model")
    f = open(File,'r', encoding="utf8")
    gloveModel = {}
    for line in f:
        splitLines = line.split()
        word = splitLines[0]
        wordEmbedding = np.array([float(value) for value in splitLines[1:]])
        gloveModel[word] = wordEmbedding
    toc = time.time()
    print(len(gloveModel)," words loaded!")
    print(f"Job took {toc-tic} seconds.")
    return gloveModel

In [8]:
glove_path = '../glove/glove.6B.300d.txt'
glove = loadGloveModel(glove_path)

Loading Glove Model
400000  words loaded!
Job took 94.98511600494385 seconds.


In [9]:
def arr_to_str(array):
    _ = ""
    for f in array:
        _ = _ + f" {f} "
    _ = _[:-1]

    return _


In [10]:
responses = amazon[~ pd.isna(amazon.in_response_to_tweet_id) & ~( amazon.inbound)]


In [11]:

x = []
y = []

tic = time.time()
for f in range(responses.iloc[:1000].shape[0]):

    in_resp_to = responses.iloc[f].in_response_to_tweet_id
    query = df[df.tweet_id == in_resp_to]
    if len(query) == 0:
        continue

    response_str = str(responses.iloc[f].text).split(" ")[2:]
    response_str = arr_to_str(response_str)

    query_str = str(query.iloc[0].text).split(" ")[2:]
    query_str = arr_to_str(query_str)
    
    y.append(response_str)
    x.append(query_str)

toc = time.time()

print(f"Job took {toc-tic} seconds")

Job took 11.927003622055054 seconds


In [16]:

def process_token(token):
    replace_list = [".", "!", ",", "?", "'", "\"", "*", ":", "[", "]"]

    for r in replace_list:
        token = token.replace(r, "")
    
    return token.lower()
    
def tokenize_set(set):

    tic = time.time()

    set_ = []
    for line in range(len(set[:])):
        tokens = set[line].split(" ")

        line = []
        for token in tokens:
            cleaned_token = process_token(token)
            try:
                num = glove[cleaned_token]
            except:
                print(f"Unknown token {token}")
                continue
            line.append(num)

        set_.append(line)

    
    set_ = np.array(set_)
    toc = time.time()
    print(f"Job took {toc-tic} seconds")

    return set_

def add_padding(x_in, y_in):

    combined = np.concatenate((x_in, y_in))
    # Padding:
    max_ = 0
    for line in combined:
        length = len(line)
        if length > max_:
            max_ = length
    
    x_out = []
    y_out = []
    for f in range(len(x)):
        # x_in[f] = np.pad(x_in[f], (0, max_ - len(x_in[f])))
        x_out.append(np.pad(x_in[f], (0, max_ - len(x_in[f]))))
        # x_in[f] = x_in[f].reshape(1, x_in[f])

    for f in range(len(y)):
        # y_in[f] = np.pad(y_in[f], (0, max_ - len(y_in[f])))
        y_out.append(np.pad(y_in[f], (0, max_ - len(y_in[f]))))
        # y_in[f] = x_in[f].reshape(1, y_in[f])

    x_out = np.array(x_out)
    y_out = np.array(y_out)
    return x_out, y_out


In [17]:

x_set = tokenize_set(x)
y_set = tokenize_set(y)


Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token #customerservice
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown t

Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token delvr
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token !!!
Unknown token 
Unknown token https://t.co/MwzaY4WoM2
Unknown token 
Unknown token ,
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token !!
Unknown token 
Unknown token 
Unknown token 
Unknown token brooo?
Unknown token 
Unknown token 🤔
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token @119625
Unknown token 
Unknown token ?
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token @115821
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 

Unknown token 
Unknown token je@amzon
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token returened
Unknown token 
Unknown token @115851
Unknown token 
Unknown token 
Unknown token @3644
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token https://t.co/QP43u5oq8o
Unknown token 
Unknown token 
Unknown token 
Unknown token @115851
Unknown token 
Unknown token @3974
Unknown token 
Unknown token 
Unknown token 
Unknown token amzon
Unknown token 
Unknown token 
Unknown token harrasing
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token https://

Unknown token 
Unknown token d'#AmazonPrimeDay
Unknown token 
Unknown token ,
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token garantir
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token envie
Unknown token 
Unknown token d'ajouter...
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token recontacter
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token avez
Unknown token 
Unknown token 
Unknown token l'humour
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token rentre
Unknown token 
Unknown token 
Unknown token l'ordre!
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unkno

Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token https://t.co/pagtQwyUf8
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token @115821
Unknown token 
Unknown token @AmazonHelp
Unknown token 

...

Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token https://t.co/TdDksLo6Mf
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token ^HD
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token https://t.co/hApLpMlfHN
Unknown token 
Unknown token ^BH
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token public.​
Unknown token 
Unknown token 3/3^EM
Unknown token 
Unknown token 
Unknown token 
Unkn

Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token phone/chat:
Unknown token 
Unknown token https://t.co/JzP7hlA23B
Unknown token 
Unknown token ^WJ
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token here:https://t.co/f9jSKbdhi3
Unknown token 
Unknown token ^FR
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token https://t.co/GIJyeYqKE0
Unknown token 
Unknown token ^MS
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token collègue
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token

Unknown token ^ZH
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token https://t.co/9mwpIdOzZn
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token ^ZH
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token ^MC
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token https://t.co/UJ5G8W6BHh
Unknown token 
Unknown token ^MH
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token uninstalling/installing
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unkn

Unknown token 
Unknown token 
Unknown token 
Unknown token deinen
Unknown token 
Unknown token #angekommen
Unknown token 
Unknown token 
Unknown token :-)
Unknown token 
Unknown token Ansonsten:
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token Verfügung.
Unknown token 
Unknown token Gruß
Unknown token 
Unknown token ^SM
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token ^NV
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token Rs.250
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 

Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token https://t.co/CLMdu2603d.
Unknown token 
Unknown token ^HA
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token ^JO
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token ^MH
Unknown token 
Unknown token should've
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token 
Unknown token

In [24]:
np.array(x_set[4]).shape

(18, 300)